In [1]:
import random
import numpy as np
import yfinance as yf
import pandas as pd
from matplotlib import pyplot as plt
from scipy.stats import norm
import math
import scipy.stats as stats
from datetime import datetime, timedelta
import statsmodels.api as sm
%matplotlib inline

In [2]:
#we assume variance is normally distributed but in real case marker is often inefficent.If marker is down today, markert return
#will remain down in the following day.so we have to recaculate volatility based on today's status.

In [3]:
tickers=['^GSPC']

In [4]:
df=yf.download(tickers,start='2014-12-31',end='2019-12-31')

[*********************100%***********************]  1 of 1 completed


In [5]:
df

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2014-12-30,2088.489990,2088.489990,2079.530029,2080.350098,2080.350098,2440280000
2014-12-31,2082.110107,2085.580078,2057.939941,2058.899902,2058.899902,2606070000
2015-01-02,2058.899902,2072.360107,2046.040039,2058.199951,2058.199951,2708700000
2015-01-05,2054.439941,2054.439941,2017.339966,2020.579956,2020.579956,3799120000
2015-01-06,2022.150024,2030.250000,1992.439941,2002.609985,2002.609985,4460110000
...,...,...,...,...,...,...
2019-12-23,3226.050049,3227.780029,3222.300049,3224.010010,3224.010010,3064530000
2019-12-24,3225.449951,3226.429932,3220.510010,3223.379883,3223.379883,1296530000
2019-12-26,3227.199951,3240.080078,3227.199951,3239.909912,3239.909912,2164540000


In [6]:
df['Return']=df['Close'].pct_change()
df['Lagged return']=df['Return'].shift(1)
df

,Open,High,Low,Close,Adj Close,Volume,Return,Lagged return
Date,,,,,,,,
2014-12-30,2088.489990,2088.489990,2079.530029,2080.350098,2080.350098,2440280000,NaN,NaN
2014-12-31,2082.110107,2085.580078,2057.939941,2058.899902,2058.899902,2606070000,-0.010311,NaN
2015-01-02,2058.899902,2072.360107,2046.040039,2058.199951,2058.199951,2708700000,-0.000340,-0.010311
2015-01-05,2054.439941,2054.439941,2017.339966,2020.579956,2020.579956,3799120000,-0.018278,-0.000340
2015-01-06,2022.150024,2030.250000,1992.439941,2002.609985,2002.609985,4460110000,-0.008893,-0.018278
...,...,...,...,...,...,...,...,...
2019-12-23,3226.050049,3227.780029,3222.300049,3224.010010,3224.010010,3064530000,0.000866,0.004945
2019-12-24,3225.449951,3226.429932,3220.510010,3223.379883,3223.379883,1296530000,-0.000195,0.000866
2019-12-26,3227.199951,3240.080078,3227.199951,3239.909912,3239.909912,2164540000,0.005128,-0.000195


In [7]:
Autocorrelation=np.correlate(df['Return'][2:],df['Lagged return'][2:])[0]#in this case our obserbation is independent bcause Autocorrelation
#is -0.001
Number_of_period=5# scaling our variance for 5 days period
variance=df['Return'][2:].var()
variance_independent=variance*Number_of_period
Autocorrelation,Number_of_period,variance,variance_independent

(-0.0018572704758445917, 5, 7.166444815888486e-05, 0.00035832224079442433)

In [8]:
weight_metrix=pd.DataFrame(columns=[i for i in range(1,6)],index=[i for i in range(1,6)])
weight_metrix

,1,2,3,4,5
1,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN


In [9]:
for i in weight_metrix.index:
    for j in weight_metrix.index:
        weight_metrix.loc[i,j]=Autocorrelation**abs(i-j)
weight_metrix

,1,2,3,4,5
1,1.0,-0.001857,0.000003,-0.0,0.0
2,-0.001857,1.0,-0.001857,0.000003,-0.0
3,0.000003,-0.001857,1.0,-0.001857,0.000003
4,-0.0,0.000003,-0.001857,1.0,-0.001857
5,0.0,-0.0,0.000003,-0.001857,1.0


In [10]:
variance_autocorrelated=variance*sum(weight_metrix.sum(axis=1))
variance_independent,variance_autocorrelated#variance_independent longterm Volatility and short term volatility for autocorrelation

(0.00035832224079442433, 0.0003572589200689124)

In [11]:
#formula for variance_autocorrelated
Scaling_factor=(1+Autocorrelation)/(1-Autocorrelation)*(Number_of_period-2*Autocorrelation*(1-Autocorrelation**Number_of_period)/(1-Autocorrelation))
Scaling_factor*variance

0.0003572584274939212

In [12]:
std_independent=np.sqrt(variance_independent)
std_autocorrelated=np.sqrt(variance_autocorrelated)
std_independent,std_autocorrelated

(0.018929401490655334, 0.018901294137410603)